#### Data Sampling

The objective of this exercise is convert to our JSON data into a tabular format. Causal modeling software usually requires a dataframe-like object (Ex: [DoWhy](https://github.com/microsoft/dowhy)). Therefore, we need to "flatten" the data in a meaningful way. 

In [2]:
import pathlib, json
import pprint as pp
import random
import pandas as pd
import matplotlib.pyplot as plt
JSON_PATH = pathlib.Path("./data/validated/")
validated = list(JSON_PATH.glob('*.json'))

To fit the data in memory, we subsample based on a few criteria. Here we have chosen year and action (e.g. "hacking")

In [3]:
#List of path objects
filtered = []
#In memory store of sampled data
content = []
#Subsampling the data

FILTER_YEAR = 2014

for entry in validated:
    with entry.open('r') as f:
        data = json.load(f)
        #Filter by action type 
        if not ('malware' in data['action'] or 'hacking' in data['action']):
            continue
        
        if data['security_incident'] != 'Confirmed':
            continue
            
        if data['timeline']['incident']['year'] < FILTER_YEAR:
            continue
        
        filtered.append(entry)
        content.append(data)
        
print("{0} entries containing hacking or malware after {1}.".format(len(filtered), FILTER_YEAR))

1144 entries containing hacking or malware after 2014.


In [4]:
# Random entry, just to give us an idea of what our data looks like.
pp.pprint(random.choice(content))

{'action': {'hacking': {'variety': ['DoS'], 'vector': ['Web application']}},
 'actor': {'external': {'country': ['Unknown'],
                        'motive': ['Unknown'],
                        'region': ['000000'],
                        'variety': ['Unknown']}},
 'asset': {'assets': [{'variety': 'S - Web application'}],
           'cloud': ['Unknown']},
 'attribute': {'availability': {'duration': {'unit': 'Hours', 'value': 2},
                                'notes': ' shut down for around two hours',
                                'variety': ['Interruption']}},
 'discovery_method': {'internal': {'variety': ['Unknown']}},
 'impact': {'overall_rating': 'Unknown'},
 'incident_id': '5EC74643-0F36-4F2A-BEBD-98A0110162C2',
 'plus': {'analysis_status': 'Finalized',
          'analyst': 'Robert-Topper',
          'created': '2016-09-22T16:54:00Z',
          'dbir_year': 2017,
          'github': '7077',
          'master_id': 'C9296FFE-AC48-4EDD-8D5C-1574E6A7F34F',
          'modified':

In [5]:
#Flattened data so far 
df = pd.io.json.json_normalize(content)

##### TODO: 
* Merge 'malware' and 'hacking' attributes

Some are categorized as both malware and hacking. Maybe look into 'variety' attribute.
* Resolve lists into categorical data


In place dropping of columns. Our list of keynames should look similar to [this](https://github.com/vz-risk/VCDB/blob/master/vcdb-keynames-real.txt)
                                                                                    

In [15]:
df = df.loc[:,~df.columns.str.startswith('value_chain')]

In [13]:
df

,incident_id,reference,schema_version,security_incident,source_id,summary,action.hacking.variety,action.hacking.vector,action.malware.notes,action.malware.variety,...,plus.pci.req_5,plus.pci.req_6,plus.pci.req_7,plus.pci.req_8,plus.pci.req_9,plus.security_maturity,actor.internal.notes,victim.notes,action.physical.notes,plus.asset.total
0,56C452C6-DC38-4BBC-A5A3-BE55C9B368D3,http://news.softpedia.com/news/Gift-Shop-PoS-a...,1.3.4,Confirmed,vcdb,Hospital Gift Shop POS system attacked,[Unknown],[Unknown],"On March 20, 2015, Saint Francis discovered th...","[Capture stored data, Capture app data]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a294a030-c495-11e7-9a8b-47d1b690bc01,http://www.workcompcentral.com/news/article/id...,1.3.4,Confirmed,vcdb,Ransomware infection,NaN,NaN,NaN,[Ransomware],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26EF258E-D5C4-4C84-B822-D5CD66B2278C,https://www.hackread.com/anonymous-breaches-th...,1.3.4,Confirmed,vcdb,Anonymous breached into the official website o...,[Unknown],[Web application],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8D72FAD3-1D68-4D7C-8BC7-6929D682600D,http://krebsonsecurity.com/2014/03/sally-beaut...,1.3.4,Confirmed,vcdb,Sally Beauty suffers data breach in their paym...,[Unknown],[Unknown],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e0d06340-0349-11e9-aee0-f7420d4f3238,http://www.austrianaviation.net/detail/hack-da...,1.3.4,Confirmed,vcdb,Aviation professionals enthusiasts have been t...,"[URL redirector abuse, Exploit vuln]",[Web application],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,0EE9C471-5D8F-4E20-9DFF-EF08DFAEDD44,http://www.universityherald.com/articles/52781...,1.3.4,Confirmed,vcdb,Carleton University in Ontario was the victim ...,[Unknown],[Unknown],NaN,[Ransomware],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,42d737a0-d919-11e7-be42-df02fe7390b5,https://krebsonsecurity.com/2017/11/hack-of-at...,1.3.4,Confirmed,vcdb,Hack of Attack-for-Hire Service vDOS Snares Ne...,[DoS],[Web application],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141,ecc26c90-81e8-11e7-ace7-c5c15a923e51,http://wspa.com/2017/04/21/spartanburg-car-was...,1.3.4,Confirmed,vcdb,Whatta Wash Car Wash said in a notice that mal...,NaN,NaN,NaN,[Spyware/Keylogger],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,282895FA-FBFF-4BA6-9D94-A3BD43E17071,https://ocrportal.hhs.gov/ocr/breach/breach_re...,1.3.4,Confirmed,vcdb,NaN,[Unknown],[Unknown],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
